# Problem Set 4

### Cristobal Cerda M.

Estimating the Brock and Mirman (1972) model by SMM 

In [3]:
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.optimize as opt
import pylab as pl
import seaborn as sns
import pandas as pd
from pandas import DataFrame, Series
import heapq
import os
import csv
import re
import math as math


1. In the next sections I will take the data from the file, generate 1000 simulations of random numbers (following a Uniform (0,1) distribution. 100 values per simulation) and define the function "normal_draws" that generate Normally distributed numbers (N($\mu$, $\sigma$)).

In [4]:
os.getcwd()

os.path.exists("data")
filestream2 = open("data/NewMacroSeries.txt", "r")

ct=[]
kt=[]
wt=[]
rt=[]
yt=[]

for line in filestream2:
    c, k, w, r, y = map(float,line.split(','))
    ct.append(c)
    kt.append(k)
    wt.append(w)
    rt.append(r)
    yt.append(y)
    
Ct=np.asarray(ct)
Kt=np.asarray(kt)
Wt=np.asarray(wt)
Rt=np.asarray(rt)
Yt=np.asarray(yt)

In [5]:
# Creating uniform draws

data_rows = 100
sim_columns = 1000
np.random.seed(1979)
uniform_mat = sts.uniform.rvs(0, 1, size=(data_rows, sim_columns))
#print(np.ndim(uniform_mat), uniform_mat.shape)

In [6]:
# Defining normal draws for error term

def normal_draws(unif_draws, mu, sigma):
    normal_draws = sts.norm.ppf(unif_draws, loc=mu, scale=sigma)
    return normal_draws

2. The next cell includes the equations of Brock & Mirman stated in the Problem Set. Each equation is developed in order to generate a simulation of one of the macroeconomic variables. 

In [7]:
# Equations of Brock & Mirman

def eq5(draws, ro, mu, sigma):
    err_draws = normal_draws(draws, 0.0, sigma)
    sim_z=np.zeros(err_draws.shape)
    i=0
    for i in range(err_draws.shape[0]):
        if i==0:
            sim_z[i,:] = err_draws[i,:]+ro*mu+(1-ro)*mu
        else:
            vec1 =sim_z[i-1,:]*ro
            vec2 = err_draws[i,:]+(1-ro)*mu
            sim_z[i,:]= vec1+vec2
    return sim_z

#print(eq5(uniform_mat,0.5,7.0,1.0)[:,599])

def eq7(k, draws, ro, mu, sigma, alpha, beta):
    err_draws = normal_draws(draws, 0.0, sigma)
    k1=k.mean()
    sim_k=np.zeros(err_draws.shape)
    i=0
    for i in range(err_draws.shape[0]):
        if i==0:
            sim_k[0,:] = k1
        else:
            vec1 = alpha*beta*np.exp(eq5(draws, ro, mu, sigma)[i,:])
            sim_k[i,:]=vec1*sim_k[i-1,:]**alpha
    return sim_k
    
#print(eq7(Kt,uniform_mat,0.5,7.0, 0.5, 0.5, 0.99)[:,599])

def eq3(k, draws, ro, mu, sigma, alpha, beta):
    err_draws = normal_draws(draws, 0.0, sigma)
    z_sim = eq5(draws, ro, mu, sigma)
    k_sim = eq7(k, draws, ro, mu, sigma, alpha, beta)
    sim_w=np.zeros(err_draws.shape)
    i=0
    for i in range(err_draws.shape[0]):
        vec1 = (1-alpha)*np.exp(z_sim[i,:])
        sim_w[i,:] = vec1*(k_sim[i,:]**alpha)
    return sim_w

#print(eq3(Kt,uniform_mat,0.5,7.0, 1.0, 0.1, 0.99))

def eq4(k, draws, ro, mu, sigma, alpha, beta):
    err_draws = normal_draws(draws, 0.0, sigma)
    z_sim = eq5(draws, ro, mu, sigma)
    k_sim = eq7(k, draws, ro, mu, sigma, alpha, beta)
    sim_r=np.zeros(err_draws.shape)
    i=0
    for i in range(err_draws.shape[0]):
                      vec1 = alpha*np.exp(z_sim[i,:])
                      sim_r[i,:] = vec1*(k_sim[i,:]**(alpha-1))
    return sim_r

#print(eq4(Kt,uniform_mat,0.5,7.0, 1.0, 0.1, 0.99))

def eq2(k, draws, ro, mu, sigma, alpha, beta):
    err_draws = normal_draws(draws, 0.0, sigma)
    z_sim = eq5(draws, ro, mu, sigma)
    k_sim = eq7(k, draws, ro, mu, sigma, alpha, beta)
    r_sim = eq4(k, draws, ro, mu, sigma, alpha, beta)
    w_sim = eq3(k, draws, ro, mu, sigma, alpha, beta)
    sim_c=np.zeros(err_draws.shape)
    i=0
    for i in range(err_draws.shape[0]):
        if i == err_draws.shape[0]-1:
            sim_c[i,:] = [x*y+z for x,y,z in zip(r_sim[i,:],k_sim[i,:],w_sim[i,:])]
        else:
            sim_c[i,:] = [x*y+z-v for x,y,z,v in zip(r_sim[i,:],k_sim[i,:],w_sim[i,:],k_sim[i+1,:])]
    return sim_c

#print(eq2(Kt,normal_draws(uniform_mat, 1, 1),0.5,1, 0.5, 0.99))

def eq6(k, draws, ro, mu, sigma, alpha, beta):
    err_draws = normal_draws(draws, 0.0, sigma)
    z_sim = eq5(draws, ro, mu, sigma)
    k_sim = eq7(k, draws, ro, mu, sigma, alpha, beta)
    sim_y=np.zeros(err_draws.shape)
    i=0
    for i in range(err_draws.shape[0]):
        sim_y[i,:] = [np.exp(x)*y**alpha for x,y in zip(z_sim[i,:],k_sim[i,:])]
    return sim_y

#print(eq6(Kt,normal_draws(uniform_mat, 1, 1),0.5,1, 0.5, 0.99))

### Part (a)

(a.1.) The functions "datamoments" and "modelmoments" generate the 6 models required in the Problem Set. both functions return a vector of the moments generated. In the case of the moments of the model, each moment is calculated for each simulation, then the 1,000 simulations of each moment are averaged.

In [8]:
# 6 moments: mean(ct), mean(kt), mean(ct/yt), var(yt), corr(ct,ct−1), corr(ct,kt)

def datamoments(variables):
    c, k, w, r, y = variables
    datamom = [0]*6 # hay un problema cuando es un array de 1D
    i=0
    c_t_1 = []
    for i in range(len(c)):
        if i==0:
            c_t_1.append(0)
        else:
            c_t_1.append(c[i-1])    
    c_t_1 = np.asarray(c_t_1)
    vec_xy = np.asarray([x/j for x,j in zip(c,y)])
    datamom[0] = c.mean()
    datamom[1] = k.mean()
    datamom[2] = vec_xy.mean()
    datamom[3] = y.var()
    datamom[5] = np.corrcoef(c[:], k[:])[0,1]
    datamom[4] = np.corrcoef(c[:], c_t_1[:])[0,1]
    return datamom
#print(datamoments(Ct, Kt, Wt, Rt, Yt))

def modelmoments(k, draws, params):
    ro, mu, sigma, alpha, beta = params
    c_sim = eq2(k, draws, ro, mu, sigma, alpha, beta)
    k_sim = eq7(k, draws, ro, mu, sigma, alpha, beta)
    w_sim = eq3(k, draws, ro, mu, sigma, alpha, beta)
    r_sim = eq4(k, draws, ro, mu, sigma, alpha, beta)
    y_sim = eq6(k, draws, ro, mu, sigma, alpha, beta)
    
    c_mean = c_sim.mean(axis=0)
    k_mean = k_sim.mean(axis=0)
    w_mean = w_sim.mean(axis=0)
    r_mean = r_sim.mean(axis=0)
    y_mean = y_sim.mean(axis=0)
    y_var = y_sim.var(axis=0)

    c_mean_2 = c_mean.mean()
    k_mean_2 = k_mean.mean()
    
    vec_xy = np.asarray(c_sim/y_sim)
    cy_mean = vec_xy.mean()
    y_var = y_var.mean()
    corr_c_k = []
    corr_c_c = []
    j=0
    c_t1 = np.zeros(c_sim.shape)
    for j in range(c_sim.shape[0]):
        if j==0:
            c_t1[j,:]=0
        else:
            c_t1[j,:]=c_sim[j-1,:]

    i=0
    for i in range(c_sim.shape[1]):
        corr_c_k.append(np.corrcoef(c_sim[:,i], k_sim[:,i])[0,1])
        corr_c_c.append(np.corrcoef(c_sim[:,i], c_t1[:,i])[0,1])
    corr_c_k = np.asarray(corr_c_k)
    corr_c_c = np.asarray(corr_c_c)
    corr_ck_mean=corr_c_k.mean()
    corr_cc_mean=corr_c_c.mean()
    
    modelmom = [0]*6
    modelmom[0] = c_mean_2
    modelmom[1] = k_mean_2
    modelmom[2] = cy_mean
    modelmom[3] = y_var
    modelmom[4] = corr_cc_mean
    modelmom[5] = corr_ck_mean
    return modelmom

#print(modelmoments(Kt,normal_draws(uniform_mat, 1, 1),0.5,1, 0.5, 0.99))

(a.2) In the next three cells I define the error vector, the criterion function and generate the minimization process.

In [9]:
def errorvec(data_vars, draws, params, simple):
    c, k, w, r, y = data_vars
    if simple:
        error_vec = [x-y for x,y in zip(modelmoments(k, draws, params),datamoments(data_vars))]
    else:
        error_vec = [(x-y)/y for x,y in zip(modelmoments(k, draws, params),datamoments(data_vars))]
    
    return error_vec

In [10]:
def criterion(params, *args):
    ro, mu, sigma, alpha = params
    beta, draws, c, k, w, r, y, W_hat = args
    data_vars = np.array([c, k, w, r, y])
    params2=np.array([ro, mu, sigma, alpha, beta])
    error_vec = np.asarray(errorvec(data_vars, draws, params2, simple = False))
    crit_value = error_vec.T@W_hat@error_vec
    return crit_value

In [11]:
ro = -0.5
mu = 7.0
alpha = 0.4
beta = 0.99
sigma = 0.7
params = np.array([ro, mu, sigma, alpha])
W_hat = np.eye(6)
draws = uniform_mat
args_smm = (beta, draws, Ct, Kt, Wt, Rt, Yt, W_hat)
result_smm = opt.minimize(criterion, params, args=(args_smm), method='L-BFGS-B', \
            bounds=((-0.99, 0.99), (5.0, 14.0), (0.01, 1.1), (0.01, 0.99)), tol = 1e-5)
ro_SMM, mu_SMM, sigma_SMM, alpha_SMM = result_smm.x
print(ro_SMM, mu_SMM, sigma_SMM, alpha_SMM)
print(result_smm)

C:\Users\Daniela\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


0.9656119642981951 6.839278773910416 0.01976221196967079 0.6007960073525342
      fun: 1.4021643729831261
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.29877296,  -1.84620463, -50.31180792, -29.09769807])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 60
      nit: 5
   status: 0
  success: True
        x: array([0.96561196, 6.83927877, 0.01976221, 0.60079601])


The solution of the minimization problem returns the following estimated parameters:

$\hat{\rho}$ = 0.95612

$\hat{\mu}$ = 6.83928

$\hat{\sigma}$ = 0.01976

$\hat{\alpha}$ = 0.60079

(a.3) The next cells include the definition of the Jacobian Matrix of the derivatives of the error vectors w.r.t each estimated parameters, in the way it is calculated in the SMM Notebook. With that I calculate the Variance-Covariance Matrix using as the weighting matrix the Identity Matrix. 

In [12]:
# Jacobian matrix of errors

def Jacobian_err(params1, params2, simple):
    ro, mu, sigma, alpha, beta = params1
    draws, c, k, w, r, y = params2
    data_vars = (Ct, Kt, Wt, Rt, Yt)
    pars = np.array([ro, mu, sigma, alpha, beta])
    datamom = np.asarray(datamoments(data_vars))
    modelmom = np.asarray(modelmoments(k, draws, pars))
    jac_mat = np.zeros((len(datamom),len(pars)))
    i=0
    epsilon=1e-5
    epsilon_vec = np.asarray([0.0]*len(pars))
    for i in range(jac_mat.shape[1]):
        epsilon_vec[i] = epsilon
        jac_mat[:,i] = (np.array(errorvec(data_vars, draws, (pars+epsilon_vec), simple)) \
                        -np.array(errorvec(data_vars, draws, (pars-epsilon_vec), simple)))/(2*epsilon)
        epsilon_vec[i] = 0
    return jac_mat


In [13]:
ro = ro_SMM
mu = mu_SMM
sigma = sigma_SMM
alpha = alpha_SMM
beta = 0.99
draws = uniform_mat
params1 = np.array([ro, mu, sigma, alpha, beta])
params2 = (draws, Ct, Kt, Wt, Rt, Yt)
simple = False
mat=Jacobian_err(params1, params2, simple)
#print(err1)
#print(err2)
#print(params1+epsilon_vec)
print(mat)
#print(modelmom)
#print(Jacobian_err(params1, params2, False))

[[ 2.53084574e-01  1.40237702e+00  9.54972292e-01  2.22776983e+01
   2.20983257e-02]
 [ 5.08071296e-01  2.80343048e+00  1.85290083e+00  4.91251759e+01
   2.83174796e+00]
 [-5.58232044e-05  8.32667268e-12 -1.92854783e-02 -1.67797889e+00
  -1.01830608e+00]
 [ 1.70576012e+00  5.35614274e-01  1.09118100e+01  9.52292394e+00
   3.34229581e-01]
 [ 3.09003054e+00  5.15164325e-01  2.08918006e+01  8.89127517e+00
  -2.48968253e-01]
 [ 2.21090258e+00  3.49878848e-02  1.86316672e+01 -2.74564060e-02
  -8.41215629e-01]]


In [14]:
S = uniform_mat.shape[1]
ro = ro_SMM
mu = mu_SMM
sigma = sigma_SMM
alpha = alpha_SMM
beta = 0.99
draws = uniform_mat
params1 = np.array([ro, mu, sigma, alpha, beta])
params2 = (draws, Ct, Kt, Wt, Rt, Yt)
simple = False
der_err = Jacobian_err(params1, params2, simple)
W_hat=np.eye(der_err.T.shape[1], der_err.T.shape[1])
#print(der_err.shape)
Var_est = (1/S)*lin.inv(der_err.T@W_hat@der_err)
ro_hat_sd = Var_est[0,0]
mu_hat_sd = Var_est[1,1]
sigma_hat_sd = Var_est[2,2]
alpha_hat_sd = Var_est[3,3]

In [15]:
print(ro_hat_sd, mu_hat_sd, sigma_hat_sd, alpha_hat_sd)

0.013802208216486431 20.304418319602977 0.0001175877654884174 0.08130456415572089


The standard deviation of the estimated parameters are the following:

$\hat{\rho}_{SD}$ = 0.01380

$\hat{\mu}_{SD}$ = 20.30441

$\hat{\sigma}_{SD}$ = 0.00012

$\hat{\alpha}_{SD}$ = 0.08130

## Part (b)

(b.1) In this section I am required to perform the estimation using the two-step estimator for the optimal weighting matrix. For that purpose, in the next cells I define the function "modelmoments" (it calculates the moments for each simulation, without averaging across simulations), the error matrix, and calculate the two-step weighting matrix.

In [16]:
# 6 moments: mean(ct), mean(kt), mean(ct/yt), var(yt), corr(ct,ct−1), corr(ct,kt)

def modelmoments2(k, draws, params):
    ro, mu, sigma, alpha, beta = params
    c_sim = eq2(k, draws, ro, mu, sigma, alpha, beta)
    k_sim = eq7(k, draws, ro, mu, sigma, alpha, beta)
    w_sim = eq3(k, draws, ro, mu, sigma, alpha, beta)
    r_sim = eq4(k, draws, ro, mu, sigma, alpha, beta)
    y_sim = eq6(k, draws, ro, mu, sigma, alpha, beta)
    
    c_mean = c_sim.mean(axis=0)
    k_mean = k_sim.mean(axis=0)
    w_mean = w_sim.mean(axis=0)
    r_mean = r_sim.mean(axis=0)
    y_mean = y_sim.mean(axis=0)
    y_var = y_sim.var(axis=0)

    cy_mean = c_mean/y_mean

    corr_c_k = []
    corr_c_c = []
    j=0
    c_t1 = np.zeros(c_sim.shape)
    for j in range(c_sim.shape[0]):
        if j==0:
            c_t1[j,:]=0
        else:
            c_t1[j,:]=c_sim[j-1,:]

    i=0
    for i in range(c_sim.shape[1]):
        corr_c_k.append(np.corrcoef(c_sim[:,i], k_sim[:,i])[0,1])
        corr_c_c.append(np.corrcoef(c_sim[:,i], c_t1[:,i])[0,1])
    corr_c_k = np.asarray(corr_c_k)
    corr_c_c = np.asarray(corr_c_c)
    model_mom_mat = np.zeros((6, draws.shape[1]))

    model_mom_mat[0,:] = c_mean
    model_mom_mat[1,:] = k_mean
    model_mom_mat[2,:] = cy_mean
    model_mom_mat[3,:] = y_var
    model_mom_mat[4,:] = corr_c_c
    model_mom_mat[5,:] = corr_c_k
    
    return model_mom_mat

In [17]:
def error_mat(data_vars, draws, params, simple):
    c, k, w, r, y = data_vars
    datamom2 = np.asarray(datamoments(data_vars))
    modelmom2 = np.asarray(modelmoments2(k, draws, params))
    error_mat = np.zeros(modelmom2.shape)
    i=0
    j=0
    if simple:
        for i in range(draws.shape[1]):
            error_mat[:,i] = modelmom2[:,i] - datamom2.T
    else:
        for j in range(draws.shape[1]):
            error_mat[:,j] = (modelmom2[:,j] - datamom2.T) / datamom2.T
    return error_mat

In [18]:
ro = ro_SMM
mu = mu_SMM
sigma = sigma_SMM
alpha = alpha_SMM
beta = 0.99
draws = uniform_mat
S = draws.shape[1]
params1 = np.array([ro, mu, sigma, alpha, beta])
variables = (Ct, Kt, Wt, Rt, Yt)
error_m = error_mat(variables, draws, params1, False)
m2 = (1/S)*error_m@error_m.T
W_h2s = lin.pinv(m2)
print(W_h2s)

[[ 6.93927227e+06 -3.47855991e+06 -1.18552871e+07  3.97973541e+03
  -1.01051100e+04  1.36579862e+04]
 [-3.47855991e+06  1.74387980e+06  5.94324718e+06 -2.10199079e+03
   4.90837045e+03 -6.58977520e+03]
 [-1.18552871e+07  5.94324718e+06  2.02579725e+07 -8.29495625e+03
   1.74372535e+04 -2.28946773e+04]
 [ 3.97973541e+03 -2.10199079e+03 -8.29495625e+03  5.86381653e+02
  -1.42714389e+02 -1.05960876e+02]
 [-1.01051100e+04  4.90837045e+03  1.74372535e+04 -1.42714389e+02
   7.23236584e+02 -8.96108978e+02]
 [ 1.36579862e+04 -6.58977520e+03 -2.28946773e+04 -1.05960876e+02
  -8.96108978e+02  1.33573758e+03]]


As in part (a), with the calculated weighting matrix, I run the minimization function, using the criterion function previously defined. 

In [19]:
ro = ro_SMM
mu = mu_SMM
sigma = sigma_SMM
alpha = alpha_SMM
beta = 0.99
params = np.array([ro, mu, sigma, alpha])
W_hat = W_h2s
draws = uniform_mat
args_smm = (beta, draws, Ct, Kt, Wt, Rt, Yt, W_hat)
result_smm = opt.minimize(criterion, params, args=(args_smm), method='L-BFGS-B', \
                          bounds=((-0.99, 0.99), (5.0, 14.0), (0.01, 0.99), (0.01, 1.1)), tol=1e-4)
ro_SMM_2, mu_SMM_2, sigma_SMM_2, alpha_SMM_2 = result_smm.x
print(ro_SMM_2, mu_SMM_2, sigma_SMM_2, alpha_SMM_2)
print(result_smm)

0.9726313826509255 6.860664948994933 0.019193704697386283 0.5995337835582838
      fun: 0.9933286063605973
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.04105349,  0.00683165, -0.44658242, -3.06029833])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 105
      nit: 15
   status: 0
  success: True
        x: array([0.97263138, 6.86066495, 0.0191937 , 0.59953378])


In this case (two-step weighting matrix), the solution of the minimization problem returns the following estimated parameters (rounded to the 5th decimal):

$\hat{\rho}$ = 0.97263

$\hat{\mu}$ = 6.8607

$\hat{\sigma}$ = 0.01919

$\hat{\alpha}$ = 0.59953

As we can see, the values of the estimated parameters do not change much regarding the previous case, neither the value of the objective function . In the next cells I will perform the calculation of the standard deviation of the estimated parameters.

In [20]:
ro = ro_SMM_2
mu = mu_SMM_2
sigma = sigma_SMM_2
alpha = alpha_SMM_2
beta = 0.99
draws = uniform_mat
params1 = np.array([ro, mu, sigma, alpha, beta])
params2 = (draws, Ct, Kt, Wt, Rt, Yt)
simple = False
mat2=Jacobian_err(params1, params2, simple)
print(mat2)

[[ 3.39704515e-01  1.40632278e+00  1.12639088e+00  2.23445341e+01
   2.20502385e-02]
 [ 6.77058977e-01  2.79712811e+00  2.18753822e+00  4.90330573e+01
   2.82538193e+00]
 [-1.98730035e-04  0.00000000e+00 -1.86702031e-02 -1.67796235e+00
  -1.01615668e+00]
 [ 2.11719269e+00  5.65692942e-01  1.20512015e+01  1.00689308e+01
   3.52095005e-01]
 [ 3.34548752e+00  5.06559681e-01  2.16206455e+01  8.75042282e+00
  -2.48903334e-01]
 [ 2.41606194e+00  3.09161833e-02  1.90385372e+01 -8.89600654e-02
  -8.37218465e-01]]


In [21]:
der_err2 = Jacobian_err(params1, params2, simple)
W_hat=W_h2s
#print(der_err.shape)
Var_est = (1/S)*lin.inv(der_err2.T@W_hat@der_err2)
ro_hat_sd = Var_est[0,0]
mu_hat_sd = Var_est[1,1]
sigma_hat_sd = Var_est[2,2]
alpha_hat_sd = Var_est[3,3]

In [22]:
print(ro_hat_sd, mu_hat_sd, sigma_hat_sd, alpha_hat_sd)

7.864106301694697e-06 0.0227853750426135 6.753483082376812e-08 9.149488758427786e-05


In this case, as it was explained in the SMM Notebook, the standard deviation of the estimated parameters is significantly lower. The results are the following:

$\hat{\rho}_{SD}$ = 7.86411e-06

$\hat{\mu}_{SD}$ = 0.02279

$\hat{\sigma}_{SD}$ = 6.75348

$\hat{\alpha}_{SD}$ = 9.14949